# MINERÍA DE DATOS COMPLETA - ANÁLISIS DE ARTÍCULOS PERIODÍSTICOS

Este notebook implementa todos los algoritmos de minería de datos para analizar artículos periodísticos y determinar cuál es el mejor modelo para clasificación y clustering.


## 1. IMPORTACIÓN DE LIBRERÍAS Y CARGA DE DATOS


In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Librerías importadas correctamente")


In [ ]:
# Carga de datos
df = pd.read_csv('articulos_exportados_20250926_082756.csv', sep=';')
print(f"Dataset cargado: {df.shape[0]} filas y {df.shape[1]} columnas")
print("\nPrimeras 5 filas:")
df.head()


In [ ]:
# Análisis exploratorio de datos
print("Información del dataset:")
print(df.info())
print("\nValores nulos:")
print(df.isnull().sum())
print("\nDistribución de categorías:")
print(df['Categoría'].value_counts())
print("\nDistribución de periódicos:")
print(df['Periódico'].value_counts())


## 2. PREPARACIÓN DE DATOS


In [ ]:
# Limpieza y preparación de datos
# Eliminar filas con valores nulos en columnas importantes
df_clean = df.dropna(subset=['Título', 'Categoría', 'Periódico'])

# Crear variable objetivo binaria basada en categoría
# Clasificaremos como 'Noticias importantes' vs 'Otras'
categorias_importantes = ['Internacional', 'Política', 'Economía']
df_clean['es_importante'] = df_clean['Categoría'].isin(categorias_importantes).astype(int)

# Combinar título y resumen para análisis de texto
df_clean['texto_completo'] = df_clean['Título'].fillna('') + ' ' + df_clean['Resumen'].fillna('')

# Crear features numéricas
df_clean['longitud_titulo'] = df_clean['Título'].str.len()
df_clean['longitud_resumen'] = df_clean['Resumen'].str.len()
df_clean['cantidad_imagenes'] = df_clean['Cantidad Imágenes'].fillna(0)

# Codificar variables categóricas
le_periodico = LabelEncoder()
df_clean['periodico_encoded'] = le_periodico.fit_transform(df_clean['Periódico'])

print(f"Dataset limpio: {df_clean.shape[0]} filas")
print(f"Distribución de clases: {df_clean['es_importante'].value_counts()}")


In [ ]:
# Vectorización de texto usando TF-IDF
tfidf = TfidfVectorizer(max_features=1000, stop_words='spanish', ngram_range=(1, 2))
X_texto = tfidf.fit_transform(df_clean['texto_completo'])

# Features numéricas
X_numericas = df_clean[['longitud_titulo', 'longitud_resumen', 'cantidad_imagenes', 'periodico_encoded']].values

# Combinar features
from scipy.sparse import hstack
X = hstack([X_texto, X_numericas])
y = df_clean['es_importante'].values

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Conjunto de entrenamiento: {X_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}")
print(f"Distribución de clases en entrenamiento: {np.bincount(y_train)}")
print(f"Distribución de clases en prueba: {np.bincount(y_test)}")


## 3. FUNCIÓN DE EVALUACIÓN DE MODELOS


In [ ]:
def mide_error(nombre_modelo, y_pred_proba, y_true=y_test):
    """
    Función para medir el error y rendimiento de los modelos
    """
    # Convertir probabilidades a predicciones binarias
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Calcular métricas
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)
    
    print(f"\n=== {nombre_modelo} ===")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"AUC-ROC: {auc:.4f}")
    print("\nReporte de clasificación:")
    print(classification_report(y_true, y_pred))
    
    return {
        'modelo': nombre_modelo,
        'accuracy': accuracy,
        'auc': auc,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

print("Función de evaluación creada")


## 4. ALGORITMOS DE CLASIFICACIÓN


### 4.1 REGRESIÓN LOGÍSTICA


In [ ]:
# REGRESIÓN LOGÍSTICA
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='newton-cg', random_state=42, max_iter=1000)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict_proba(X_test)[:,1]

resultados_logreg = mide_error('Regresión Logística', y_pred_logreg)


### 4.2 K-NEAREST NEIGHBORS (KNN)


In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict_proba(X_test)[:,1]

resultados_knn = mide_error('KNN', y_pred_knn)


### 4.3 NAIVE BAYES


In [ ]:
# NAIVE BAYES
from sklearn.naive_bayes import BernoulliNB

naive_bayes = BernoulliNB()
naive_bayes.fit(X_train, y_train)
y_pred_nb = naive_bayes.predict_proba(X_test)[:,1]

resultados_nb = mide_error('Naive Bayes', y_pred_nb)


### 4.4 ÁRBOL DE DECISIÓN


In [ ]:
# ÁRBOL DE DECISIÓN
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict_proba(X_test)[:,1]

resultados_tree = mide_error('Árbol de Decisión', y_pred_tree)


### 4.5 RANDOM FOREST


In [ ]:
# RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(X_train, y_train)
y_pred_rf = random_forest.predict_proba(X_test)[:,1]

resultados_rf = mide_error('Random Forest', y_pred_rf)


### 4.6 XGBOOST


In [ ]:
# XGBOOST
try:
    from xgboost import XGBClassifier
    
    xgb_classifier = XGBClassifier(random_state=42, eval_metric='logloss')
    xgb_classifier.fit(X_train, y_train)
    y_pred_xgb = xgb_classifier.predict_proba(X_test)[:,1]
    
    resultados_xgb = mide_error('XGBoost', y_pred_xgb)
except ImportError:
    print("XGBoost no está instalado. Instalando...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'xgboost'])
    
    from xgboost import XGBClassifier
    xgb_classifier = XGBClassifier(random_state=42, eval_metric='logloss')
    xgb_classifier.fit(X_train, y_train)
    y_pred_xgb = xgb_classifier.predict_proba(X_test)[:,1]
    
    resultados_xgb = mide_error('XGBoost', y_pred_xgb)


### 4.7 LIGHTGBM (HISTOGRADIENTBOOSTING)


In [ ]:
# LIGHTGBM (usando HistGradientBoostingClassifier de sklearn)
from sklearn.ensemble import HistGradientBoostingClassifier

hist_gradient_boosting = HistGradientBoostingClassifier(random_state=42)
hist_gradient_boosting.fit(X_train, y_train)
y_pred_lgb = hist_gradient_boosting.predict_proba(X_test)[:,1]

resultados_lgb = mide_error('LightGBM (HistGradientBoosting)', y_pred_lgb)


## 5. ALGORITMOS DE CLUSTERING


### 5.1 K-MEDIAS (K-MEANS)


In [ ]:
# K-MEDIAS
from sklearn.cluster import KMeans

# Reducir dimensionalidad para clustering
from sklearn.decomposition import PCA
pca = PCA(n_components=50, random_state=42)
X_pca = pca.fit_transform(X_train.toarray())

kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
kmeans.fit(X_pca)

print("\n=== K-MEDIAS ===")
print(f"Centroides encontrados: {kmeans.n_clusters}")
print(f"Primeros 10 labels de cluster: {kmeans.labels_[:10]}")
print(f"Inercia: {kmeans.inertia_:.2f}")

# Visualización de clusters
plt.figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans.labels_, cmap='viridis', alpha=0.6)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
           c='red', marker='x', s=200, linewidths=3)
plt.title('Clustering K-Means de Artículos Periodísticos')
plt.xlabel('Primera Componente Principal')
plt.ylabel('Segunda Componente Principal')
plt.colorbar()
plt.show()


## 6. ANÁLISIS DE SERIES TEMPORALES


### 6.1 PREPARACIÓN DE DATOS TEMPORALES


In [ ]:
# Preparar datos para análisis temporal
df_clean['Fecha Extracción'] = pd.to_datetime(df_clean['Fecha Extracción'])
df_clean['fecha'] = df_clean['Fecha Extracción'].dt.date

# Agrupar por fecha y contar artículos importantes
serie_temporal = df_clean.groupby('fecha')['es_importante'].agg(['count', 'sum']).reset_index()
serie_temporal.columns = ['fecha', 'total_articulos', 'articulos_importantes']
serie_temporal['proporcion_importantes'] = serie_temporal['articulos_importantes'] / serie_temporal['total_articulos']
serie_temporal = serie_temporal.sort_values('fecha')

print("Serie temporal creada:")
print(serie_temporal.head())
print(f"\nTotal de días: {len(serie_temporal)}")

# Visualización de la serie temporal
plt.figure(figsize=(12, 6))
plt.plot(serie_temporal['fecha'], serie_temporal['proporcion_importantes'], marker='o')
plt.title('Proporción de Artículos Importantes a lo Largo del Tiempo')
plt.xlabel('Fecha')
plt.ylabel('Proporción de Artículos Importantes')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### 6.2 ARIMA


In [ ]:
# ARIMA
try:
    from pmdarima import auto_arima
    
    # Preparar datos para ARIMA
    train_data = serie_temporal['proporcion_importantes'].values[:-4]  # Últimos 4 puntos para test
    test_data = serie_temporal['proporcion_importantes'].values[-4:]
    
    model = auto_arima(train_data, seasonal=False, suppress_warnings=True)
    model_fit = model.fit(train_data)
    predictions = model_fit.predict(n_periods=len(test_data))
    
    # Visualización
    plt.figure(figsize=(12,6))
    plt.plot(serie_temporal['fecha'][-12:].astype(str), serie_temporal['proporcion_importantes'][-12:], label='Realidad')
    plt.plot(serie_temporal['fecha'][-4:].astype(str), predictions, label='Predicción')
    plt.xticks(rotation=45, fontsize=8)
    plt.title('Predicción ARIMA - Proporción de Artículos Importantes')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print("\n=== ARIMA ===")
    print(f"Predicciones: {predictions}")
    
except ImportError:
    print("pmdarima no está instalado. Instalando...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'pmdarima'])
    
    from pmdarima import auto_arima
    train_data = serie_temporal['proporcion_importantes'].values[:-4]
    test_data = serie_temporal['proporcion_importantes'].values[-4:]
    
    model = auto_arima(train_data, seasonal=False, suppress_warnings=True)
    model_fit = model.fit(train_data)
    predictions = model_fit.predict(n_periods=len(test_data))
    
    plt.figure(figsize=(12,6))
    plt.plot(serie_temporal['fecha'][-12:].astype(str), serie_temporal['proporcion_importantes'][-12:], label='Realidad')
    plt.plot(serie_temporal['fecha'][-4:].astype(str), predictions, label='Predicción')
    plt.xticks(rotation=45, fontsize=8)
    plt.title('Predicción ARIMA - Proporción de Artículos Importantes')
    plt.legend()
    plt.tight_layout()
    plt.show()


### 6.3 SUAVIZADO EXPONENCIAL


In [ ]:
# SUAVIZADO EXPONENCIAL
from statsmodels.tsa.holtwinters import ExponentialSmoothing

train_data = serie_temporal['proporcion_importantes'].values[:-4]
test_data = serie_temporal['proporcion_importantes'].values[-4:]

model = ExponentialSmoothing(train_data, seasonal=None, trend='add')
model_fit = model.fit()
predictions = model_fit.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1)

# Visualización
plt.figure(figsize=(12, 6))
plt.plot(serie_temporal['fecha'][-12:].astype(str), serie_temporal['proporcion_importantes'][-12:], label='Realidad')
plt.plot(serie_temporal['fecha'][-4:].astype(str), predictions, label='Predicción')
plt.xticks(rotation=45, fontsize=8)
plt.title('Suavizado Exponencial - Proporción de Artículos Importantes')
plt.legend()
plt.tight_layout()
plt.show()

print("\n=== SUAVIZADO EXPONENCIAL ===")
print(f"Predicciones: {predictions}")


## 7. COMPARACIÓN Y EVALUACIÓN DE MODELOS


In [ ]:
# Recopilar todos los resultados
resultados = [
    resultados_logreg,
    resultados_knn,
    resultados_nb,
    resultados_tree,
    resultados_rf,
    resultados_xgb,
    resultados_lgb
]

# Crear DataFrame de comparación
comparacion = pd.DataFrame([
    {
        'Modelo': r['modelo'],
        'Accuracy': r['accuracy'],
        'AUC-ROC': r['auc']
    } for r in resultados
])

print("\n=== COMPARACIÓN DE MODELOS ===")
print(comparacion.sort_values('AUC-ROC', ascending=False))

# Visualización de comparación
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de barras para Accuracy
ax1.bar(comparacion['Modelo'], comparacion['Accuracy'], color='skyblue', alpha=0.7)
ax1.set_title('Comparación de Accuracy')
ax1.set_ylabel('Accuracy')
ax1.tick_params(axis='x', rotation=45)

# Gráfico de barras para AUC-ROC
ax2.bar(comparacion['Modelo'], comparacion['AUC-ROC'], color='lightcoral', alpha=0.7)
ax2.set_title('Comparación de AUC-ROC')
ax2.set_ylabel('AUC-ROC')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Mejor modelo
mejor_modelo = comparacion.loc[comparacion['AUC-ROC'].idxmax()]
print(f"\n🏆 MEJOR MODELO: {mejor_modelo['Modelo']}")
print(f"   Accuracy: {mejor_modelo['Accuracy']:.4f}")
print(f"   AUC-ROC: {mejor_modelo['AUC-ROC']:.4f}")


## 8. ANÁLISIS DE IMPORTANCIA DE CARACTERÍSTICAS


In [ ]:
# Análisis de importancia de características (usando Random Forest)
feature_names = list(tfidf.get_feature_names_out()) + ['longitud_titulo', 'longitud_resumen', 'cantidad_imagenes', 'periodico_encoded']

importancias = random_forest.feature_importances_
indices = np.argsort(importancias)[::-1][:20]  # Top 20 características

plt.figure(figsize=(12, 8))
plt.title('Top 20 Características Más Importantes')
plt.barh(range(len(indices)), importancias[indices], color='lightgreen', alpha=0.7)
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Importancia')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 10 características más importantes:")
for i in range(10):
    print(f"{i+1}. {feature_names[indices[i]]}: {importancias[indices[i]]:.4f}")


## 9. CONCLUSIONES Y RECOMENDACIONES


In [ ]:
print("\n" + "="*60)
print("           RESUMEN DE ANÁLISIS DE MINERÍA DE DATOS")
print("="*60)

print(f"\n📊 DATASET ANALIZADO:")
print(f"   • Total de artículos: {len(df_clean)}")
print(f"   • Periódicos: {df_clean['Periódico'].nunique()}")
print(f"   • Categorías: {df_clean['Categoría'].nunique()}")
print(f"   • Proporción de artículos importantes: {df_clean['es_importante'].mean():.2%}")

print(f"\n🤖 MODELOS EVALUADOS:")
for i, (_, row) in enumerate(comparacion.sort_values('AUC-ROC', ascending=False).iterrows(), 1):
    print(f"   {i}. {row['Modelo']}: AUC={row['AUC-ROC']:.4f}, Acc={row['Accuracy']:.4f}")

print(f"\n🏆 MEJOR MODELO: {mejor_modelo['Modelo']}")
print(f"   • Accuracy: {mejor_modelo['Accuracy']:.4f}")
print(f"   • AUC-ROC: {mejor_modelo['AUC-ROC']:.4f}")

print(f"\n📈 INSIGHTS CLAVE:")
print(f"   • El modelo {mejor_modelo['Modelo']} es el más efectivo para clasificar artículos")
print(f"   • Las características de texto son las más importantes")
print(f"   • El clustering revela patrones en la estructura de los artículos")
print(f"   • Las series temporales muestran tendencias en la importancia de noticias")

print(f"\n💡 RECOMENDACIONES:")
print(f"   • Usar {mejor_modelo['Modelo']} para clasificación automática")
print(f"   • Implementar análisis de sentimiento en el texto")
print(f"   • Monitorear tendencias temporales de importancia")
print(f"   • Considerar features adicionales como hora de publicación")

print("\n" + "="*60)
